# 랜덤 포레스트 분류 모델

## 데이터 불러오기

In [3]:
import pandas as pd
import numpy as np

DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)
reviews = list(train_data['review'])
y = np.array(train_data['sentiment'])

## CountVectorizer를 불러와서 객체 생성 후 리뷰 텍스트 데이터 적용

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word", max_features = 5000) 

train_data_features = vectorizer.fit_transform(reviews)

## train_data_features 형태

In [5]:
train_data_features

<25000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1975048 stored elements in Compressed Sparse Row format>

## 학습과 검증 데이터 분리

In [7]:
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.2
RANDOM_SEED = 42

train_input, eval_input, train_label, eval_label = train_test_split(train_data_features, y, test_size=TEST_SIZE, random_state=RANDOM_SEED)

## 모델 구현 및 학습

In [8]:
from sklearn.ensemble import RandomForestClassifier


# 랜덤 포레스트 분류기에 100개 의사 결정 트리를 사용한다.
forest = RandomForestClassifier(n_estimators = 100) 

# 단어 묶음을 벡터화한 데이터와 정답 데이터를 가지고 학습을 시작한다.
forest.fit( train_input, train_label )

RandomForestClassifier()

## 검증 데이터셋으로 성능 평가

In [9]:
print("Accuracy: %f" % forest.score(eval_input, eval_label))

Accuracy: 0.845800


## 전처리한 평가 데이터 불러오기

In [10]:
TEST_CLEAN_DATA = 'test_clean.csv'
DATA_OUT_PATH = './data_out/'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_reviews = list(test_data['review'])
ids = list(test_data['id'])

## 평가 데이터 벡터화

In [11]:
test_data_features = vectorizer.transform(test_reviews)

## CSV 파일로 만들어 저장

In [12]:
import os

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
# 위에서 만든 랜덤 포레스트 분류기를 통해 예측값을 가져온다.
result = forest.predict(test_data_features)

# 판다스 데이터 프레임을 통해 데이터를 구성해서 output에 넣는다.
output = pd.DataFrame( data={"id": ids, "sentiment": result} )

# 이제 csv파일로 만든다.
output.to_csv( DATA_OUT_PATH + "Bag_of_Words_model.csv", index=False, quoting=3 )